In [ ]:
# warn about refactor
import warnings
warnings.warn("Use the refactored 'screpairutils' module instead")

### Literals

*regarding input files*

In [ ]:
# these library types specify which data we can expect per row
DAMID_LIBTYPES = {"DamID", "ChICandDamID", "DamIDandT", "Damaris"}
CHIC_LIBTYPES = {"ChIC", "ChICandDamID"}
CELSEQ_LIBTYPES = {"DamIDandT"}
DAMARIS_LIBTYPES = {"Damaris"}

In [ ]:

DAMID_POS_FNFMT = "./experiments/{datadir}/data/counts/{limsid}.index{indexnr:02d}.{barcode}.event_counts.pos.hdf5"
DAMID_BINNED_FNFMT = "./experiments/{datadir}/data/counts/{limsid}.index{indexnr:02d}.{barcode}.event_counts.binsize_{binsize:d}.hdf5"
DAMID_BINNED_FNFMT_OLDPOP = "./experiments/{datadir}/data/counts/{limsid}.index{indexnr:02d}.{barcode}.counts.binsize_{binsize:d}.hdf5"
DAMID_OLDPOPFMT_LIMSIDS = {"KIN1554", "KIN1555", "KIN1583", "KIN1726"}

TX_FNFMT = "./experiments/{datadir}/data/counts/{limsid}.index{indexnr:02d}.{barcode}.counts.hdf5"

DAMARIS_FNFMT = "./experiments/{datadir}/data/counts/{limsid}.index{indexnr:02d}.{barcode}.invalid_pos_reads.counts.hdf5"

# try multiple, in order:
CHIC_BINNED_FNFMTS = [
    "./experiments/{datadir}/data/counts/{limsid}.index{indexnr:02d}.{barcode}.chic.event_counts.binsize_{binsize:d}.hdf5",
    "./experiments/{datadir}/data/counts/{limsid}.index{indexnr:02d}.{barcode}.chic.counts.binsize_{binsize:d}.hdf5",
]

In [ ]:
BARCODENAMEFMTS = {
    "unspecified": "BC_{barcodenr:03d}",
    "damid": "DamID_BC_{barcodenr:03d}",
    "damid2": "DamID2_BC_{barcodenr:03d}",
    "damid_v2": "BC_DamIDv2_{barcodenr:03d}",
    "damid_v3": "BC_DamIDv3_{barcodenr:03d}",
    "damid_v3_set1": "BCv3set1_BC_{barcodenr:03d}",
    "damid_v3_set2": "BCv3set2_BC_{barcodenr:03d}",
    "chic": "BC_ChIC_{barcodenr:03d}",
}

### Get dataset from HDF5

In [ ]:
def get_dataset_with_dim(f, name, shape, dtype=None):
    if not name in f:
        return np.zeros(shape, dtype=(int if dtype is None else dtype))
    else:
        d = f[name][:]
        if d.shape == shape:
            if dtype is None:
                return d
            else:
                return d.astype(dtype)

        if len(shape) > d.ndim:
            raise ValueError("Could be implemented with `np.pad` but I'm lazy")

        if d.ndim > len(shape):
            reduceaxes = d.ndim - len(shape)
            d = np.add.reduce(d, -reduceaxes)

        diff = shape[0] - len(d)
        if diff > 0:
            d = np.pad(d, (0, diff), mode='constant', constant_values=d.dtype.type())
        elif diff < 0:
            d = d[:shape[0]]

        return d

### Scaling factor

In [ ]:
import warnings

import numpy as np
import scipy.signal
import scipy.stats

In [ ]:
def normalized_gaussian_window(s):
    s = float(s)
    W = scipy.signal.windows.gaussian(int(np.ceil(3 * s)) * 2 + 1, s)
    return (W / W.sum())

In [ ]:
SMOOTH_STDDEV_SF = 2500e3

W_sf = normalized_gaussian_window(SMOOTH_STDDEV_SF / BINSIZE)

In [ ]:
# pre-convolve mappability for the sake of SF calculations
binned_mapab_c = {
    chrom: scipy.signal.fftconvolve(binned_mapab[chrom].astype(float), W_sf, mode='same')
    for chrom in chroms
}

total_mapab = sum(int(binned_mapab[chrom].sum()) for chrom in chroms)

In [ ]:
def calc_scale_factor(X, alpha=0.5):
    """
    X: (Nbins, Nsamples)
    """
    
    _offset = 0.01
    
    assert X.ndim == 2
    assert (X.sum(axis=0) > 0).all()
    
    w = ((~np.isclose(X, 0)) & (X > 0)).any(axis=1)
    assert w.ndim == 1
    
    assert w.sum() > 0
    
    Xij = X[w]
    S = np.ones(len(Xij), dtype=bool)
    
    Xi = Xij.sum(axis=1)
    sj = Xij[S].sum(axis=0) / Xi[S].sum(axis=0)
    
    assert np.isclose(sj.sum(), 1.)
    
    Eij = np.outer(Xi, sj)
    GOFi = (((Xij - Eij) ** 2) / Eij).sum(axis=1)
    assert len(GOFi) == len(S)
    
    _GOF_low, _GOF_high = np.quantile(GOFi, np.array([_offset, min(1.0 - _offset, 1 - alpha + _offset)]))
    
    for _ in range(20):
        S_update = (GOFi >= _GOF_low) & (GOFi < _GOF_high)
        if (S == S_update).all():
            break

        S = S_update.copy()

        sj = Xij[S].sum(axis=0) / Xi[S].sum(axis=0)
        
        Eij = np.outer(Xi, sj)
        GOFi = (((Xij - Eij) ** 2) / Eij).sum(axis=1)
        
        _GOF_low, _GOF_high = np.quantile(GOFi, np.array([_offset, min(1.0 - _offset, 1 - alpha + _offset)]))
    else:
        warnings.warn("Did not converge")
    
    sj_final = sj.copy()
    
    return sj_final

In [ ]:
def calc_scale_factor_adapt(X, alpha=0.5):
    """
    X: (Nbins, Nsamples)
    """
    
    _offset = 0.01
    
    assert X.ndim == 2
    assert (X.sum(axis=0) > 0).all()
    
    w = ((~np.isclose(X, 0)) & (X > 0)).any(axis=1)
    assert w.ndim == 1
    
    assert w.sum() > 0
    
    Xij = X[w]
    S = np.ones(len(Xij), dtype=bool)
    
    Xi = Xij.sum(axis=1)
    sj = Xij[S].sum(axis=0) / Xi[S].sum(axis=0)
    
    assert np.isclose(sj.sum(), 1.)
    
    Eij = np.outer(Xi, sj)
    # Note: difference instead of squared distance!
    GOFi = ((Xij - Eij) / Eij).sum(axis=1)
    assert len(GOFi) == len(S)
    
    _GOF_low, _GOF_high = np.quantile(GOFi, np.array([0.01, min(1.0 - 0.01, 1 - alpha + 0.01)]))
    
    for _ in range(20):
        S_update = (GOFi >= _GOF_low) & (GOFi < _GOF_high)
        if (S == S_update).all():
            break

        S = S_update.copy()
        sj = Xij[S].sum(axis=0) / Xi[S].sum(axis=0)
        
        Eij = np.outer(Xi, sj)
        GOFi = (((Xij - Eij) ** 1) / Eij).sum(axis=1)

        _GOF_low, _GOF_high = np.quantile(GOFi, np.array([0.01, min(1.0 - 0.01, 1 - alpha + 0.01)]))
    else:
        warnings.warn("Did not converge")
    
    sj_final = sj.copy()
    
    return sj_final

In [ ]:
# wrapper function:
def calc_sf(fg, bg, w_mapab, alpha):
    total_bg = float(sum(v.sum() for v in bg.values()))
    assert fg.keys() == bg.keys()
    assert set(w_mapab.keys()).issuperset(set(fg.keys()))
    
    chroms = sorted(fg.keys())
    
    fg_c = {chrom: scipy.signal.fftconvolve(fg[chrom].astype(float), W_sf, mode='same') for chrom in chroms}
    bg_c = {chrom: scipy.signal.fftconvolve(bg[chrom].astype(float), W_sf, mode='same') for chrom in chroms}
    
    Xij = np.array([
        np.concatenate([fg_c[chrom][w_mapab[chrom]] for chrom in chroms]),
        np.concatenate([bg_c[chrom][w_mapab[chrom]] for chrom in chroms]),
    ]).T

    with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        rel_sf = calc_scale_factor(Xij, alpha=alpha)

    assert np.isclose(1., rel_sf.sum())

    sf = rel_sf[0] / (rel_sf[1] / total_bg)  # scale fg to bg
    assert sf > 0.
    
    return sf

difference, distance (absolute difference) and squared, visualized:

In [ ]:
# %matplotlib inline

# from matplotlib import pyplot as plt

# from sklearn.preprocessing import minmax_scale

In [ ]:
# a, b = np.meshgrid(np.arange(1, 10), np.arange(1, 5))

In [ ]:
# y = (a - b) / b

# plt.pcolormesh(minmax_scale(y.ravel()).reshape(y.shape), cmap="coolwarm", vmin=0, vmax=1)

In [ ]:
# y = np.abs(a - b) / b

# plt.pcolormesh(minmax_scale(y.ravel()).reshape(y.shape), cmap="coolwarm", vmin=0, vmax=1)

In [ ]:
# y = ((a - b) ** 2) / b

# plt.pcolormesh(minmax_scale(y.ravel()).reshape(y.shape), cmap="coolwarm", vmin=0, vmax=1)

## Load AsiSI site tables

In [ ]:
ASISI_SITE_FN = "/data/zfs/deepseq/projects/DNADamage/KR20160411/data/regions/Homo_sapiens.GRCh37.dna.primary_assembly.AsiSI_GCGATCGC.bed"

TOP_SITE_FN = "/data/zfs/deepseq/projects/DNADamage/KR20170201.AsiSI_top_sites/output/KR20170202.write_breakseq_chipseq_defined_asisi_tophit_subset/KR20170203.BREAkseq_and_gH2AX_defined_AsiSI_top_sites.tsv"

In [ ]:
asisi_sites = pd.read_csv(ASISI_SITE_FN, sep="\t", header=None, names=["chrom", "start", "end"])
# NOTE: asisi sites are also listed on contigs (GL.. etc); remove those entries
asisi_sites = asisi_sites[asisi_sites["chrom"].isin(chroms)].reset_index(drop=True)

top_sites = pd.read_csv(TOP_SITE_FN, sep="\t", header=None, usecols=[0, 1, 2], names=["chrom", "start", "end"])

assert ((asisi_sites["end"] - asisi_sites["start"]) == 8).all()

assert ((top_sites["end"] - top_sites["start"]) == 8).all()

## Subsampling

In [ ]:
def subsample_prob(v, n, random_seed=None):
    """
    Probablistic subsampling
    """
    assert v.ndim == 1
    vs = v.sum()
    assert n <= vs
    rs = np.random.RandomState(random_seed)
    return rs.binomial(n, v / vs)


def subsample_chroms_prob(ds, n, random_seed=None):
    # transparently concatenate chroms and re-split after subsampling
    dschroms = sorted(ds)
    v = np.concatenate([ds[chrom] for chrom in dschroms])
    vs = subsample_prob(v, n, random_seed)
    dschrompos = np.array([ds[chrom].size for chrom in dschroms]).cumsum()[:-1]
    return dict(zip(dschroms, np.split(vs, dschrompos)))

## Data loading routines

In [ ]:
# def negate_chroms(vs):
#     return {chrom: ~vs[chrom] for chrom in vs}

# embrace the functional approach:
from functools import reduce

def map_dict(func, *args):
    keys = reduce(set.intersection, (set(d.keys()) for d in args))
    return {k: func(*[d[k] for d in args]) for k in keys}
# --> negate_chroms(vs) =~ map_dict(np.logical_not, vs)

In [ ]:
def mask_ds(ds, w, fillvalue=0):
    return np.choose(
        w,
        (ds, fillvalue)
    )

In [ ]:
def barcode_from_row(row):
    return BARCODENAMEFMTS[row["genomic_barcodetype"]].format(barcodenr=row["barcodenr"])


def damid_fn_from_row(row):
    barcode = barcode_from_row(row)
    
    if (
        (row["limsid"] in DAMID_OLDPOPFMT_LIMSIDS)
        and (row["cellcount"] > 16)
    ):
        fn = DAMID_BINNED_FNFMT_OLDPOP.format(**row.to_dict(), barcode=barcode, binsize=BINSIZE)
    else:
        fn = DAMID_BINNED_FNFMT.format(**row.to_dict(), barcode=barcode, binsize=BINSIZE)
    
    return fn


def chic_fn_from_row(row):
    barcode = barcode_from_row(row)
    
    # deal with multiple output file formats :/
    fns = map(
        lambda fnfmt: fnfmt.format(**row.to_dict(), barcode=barcode, binsize=BINSIZE),
        CHIC_BINNED_FNFMTS,
    )
    fn = next(fn for fn in fns if os.access(fn, os.R_OK))
    
    return fn

In [ ]:
def load_binned_ds(fn, binned_chromsizes):
    chroms = binned_chromsizes.keys()
    
    with h5py.File(fn, 'r') as f:
        ds = {
            chrom: get_dataset_with_dim(f, chrom, (binned_chromsizes[chrom], ))
            for chrom in chroms
        }
        
    return ds

---